In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
# setup database connection
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'fb13'))
sess = driver.session()

In [3]:
def res_to_df(res):
    result_list = [record for record in res]
    return pd.DataFrame(result_list, columns=res.keys())  

In [7]:
q = '''
MATCH (a:Person)<-[:children]-(x:Person)<-[:parents]-(b:Person)
WHERE a <> b
RETURN a.name as child, b.name as sibling, x.name as parent

UNION

MATCH (a:Person)-[:parents]->(x:Person)<-[:parents]-(b:Person)
WHERE a <> b
RETURN a.name as child, b.name as sibling, x.name as parent

UNION

MATCH (a:Person)<-[:children]-(x:Person)-[:children]->(b:Person)
WHERE a <> b
RETURN a.name as child, b.name as sibling, x.name as parent

UNION

MATCH (a:Person)-[:parents]->(x:Person)-[:children]->(b:Person)
WHERE a <> b
RETURN a.name as child, b.name as sibling, x.name as parent
'''
family_triples = res_to_df(sess.run(q))

In [40]:
num_shared_parents = pd.DataFrame(family_triples.groupby(['child', 'sibling'], as_index=False).size())
num_shared_parents.columns = ['child', 'sibling', 'num_parents']

In [44]:
num_shared_parents.shape
num_shared_parents[num_shared_parents.num_parents > 1].shape

(8252, 3)

(4408, 3)

In [33]:
x[x==2]

child                        sibling                    
abigail_may_alcott_nieriker  louisa_may_alcott              2
adela_of_normandy            henry_i_of_england             2
                             robert_iii_duke_of_normandy    2
                             william_ii_of_england          2
adelaide_rixa_of_poland      casimir_i_of_poland            2
                                                           ..
zeppo_marx                   chico_marx                     2
                             groucho_marx                   2
                             gummo_marx                     2
                             harpo_marx                     2
zita_of_bourbon-parma        felix_of_bourbon-parma         2
Length: 4388, dtype: int64

In [8]:
sib1 = family_triples.groupby('child').agg(set)

In [11]:
for sib in sib1:
    print(sib)

sibling
parent


In [21]:
person = 'william_ii_of_england'
P = sib1['parent']
P[person]

S = sib1['sibling']
S[person]

{'matilda_of_flanders', 'william_i_of_england'}

{'adela_of_normandy', 'henry_i_of_england', 'robert_iii_duke_of_normandy'}

{'matilda_of_flanders', 'william_i_of_england'}

In [12]:
family_triples.head()

,child,sibling,parent
0,frederick_i_margrave_of_brandenburg-ansbach,john_elector_of_brandenburg,albert_iii_elector_of_brandenburg
1,harry_brodribb_irving,laurence_irving,henry_irving
2,grand_duke_alexander_alexandrovich_of_russia,nicholas_ii_of_russia,alexander_iii_of_russia
3,grand_duke_alexander_alexandrovich_of_russia,grand_duke_michael_alexandrovich_of_russia,alexander_iii_of_russia
4,grand_duke_alexander_alexandrovich_of_russia,grand_duchess_olga_alexandrovna_of_russia,alexander_iii_of_russia
